Import used packages

In [ ]:
import cv2
import os
import numpy as np
from algorithms import *

Define image path

In [ ]:
image_path = './assets/low2.jpg'
file_extension = os.path.splitext(image_path)[1]
image_type = file_extension[1:].lower()

Load RGB image

In [ ]:
# Load the image in color format
img = cv2.imread(image_path, cv2.IMREAD_COLOR)

Change image to matrix

In [ ]:
matrix = np.float32(img)

# Print the matrix shape
print("Matrix shape:", matrix.shape)

In [ ]:
original_size = save_image(matrix, image_type, onlySize=True)
print(f'{original_size} MB')

Split the image into three separate color channels

In [ ]:
# Slice the third dimension of the array

b = matrix[:,:,0]
g = matrix[:,:,1]
r = matrix[:,:,2]

Apply SVD to each channel

In [ ]:
U_b, S_b, Vt_b = svd(b)
U_g, S_g, Vt_g = svd(g)
U_r, S_r, Vt_r = svd(r)

Find the Number of singular Values

In [ ]:
num_sv = 50

Trunctate the matrices U, S, V

In [ ]:
U_b_truncate, S_b_truncate, Vt_b_truncate = truncate(U_b, S_b, Vt_b, num_sv)
U_g_truncate, S_g_truncate, Vt_g_truncate = truncate(U_g, S_g, Vt_g, num_sv)
U_r_truncate, S_r_truncate, Vt_r_truncate = truncate(U_r, S_r, Vt_r, num_sv)